In [1]:
import os
import scanpy as sc
import pandas as pd

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from scipy.sparse import issparse
import matplotlib.pyplot as plt
import seaborn as sns

import gseapy as gp


In [2]:
import sys
sys.path.append('../../')

from downstream.svg import detect_svg
from downstream.plotting import svgplot,gheatmap

import utils

# Config

In [3]:
# set global rc parameters
sc.settings.set_figure_params(vector_friendly=True)

utils.config_rc(dpi=300, font_size=6)
# mpl.rcParams


In [4]:
wk_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Get SVG

In [5]:
# stereo-seq mouse embryo data result root path
data_root = utils.get_path('svg_wk_root', json_path=os.path.join(wk_root, '_data.json'))

df_group=pd.read_csv(os.path.join(data_root, 'mouse_embryo/svg_ME_All_Domain_Filtered_0.75_8.csv'))
df_group=df_group[~df_group['gene'].str.startswith('mt-')].copy()

In [1]:
adata = sc.read_h5ad(os.path.join(data_root, 'mouse_embryo/preprocessed_ME.h5ad'))
SpaSEG_cluster = pd.read_csv(os.path.join(data_root, 'mouse_embryo/ME_bin50_metadata.tsv'),sep = '\t')

adata.obs['SpaSEG_clusters'] = list(SpaSEG_cluster['SpaSEG_clusters'])
adata.obs['SpaSEG_clusters'] = adata.obs['SpaSEG_clusters'].astype('category')
adata.uns['log1p'] = {'base': None}

adata.obsm['spatial'] = adata.obsm['spatial'][:,[1,0]]*-1
adata.obsm['spatial'][:,[0]] = adata.obsm['spatial'][:,[0]]*-1

In [8]:
adata

AnnData object with n_obs × n_vars = 69299 × 26921
    obs: 'index', 'SCT_snn_res.0.6', 'SCT_snn_res.0.8', 'SCT_snn_res.1', 'annotation', 'annotation2', 'coor_x', 'coor_y', 'lineage', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'orig.ident', 'percent.mt', 'primaryanno', 'refindedanno', 'seurat_clusters', 'simpleanno', 'spatial_leiden_res1', 'spatial_leiden_res2', 'spatial_leiden_res3', 'annotation3', 'chip', 'batch', 'simpleanno2', 'S_score', 'G2M_score', 'phase', 'n_counts', 'annotation4', 'array_col', 'array_row', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_genes', 'SpaSEG_clusters'
    var: 'n_cells-2', 'n_cells-3', 'n_cells-4', 'n_cells-5', 'n_cells-6', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', '

# GO

In [6]:
_group_df =  df_group.sort_values(by=['domain','log2_fc'],ascending=[True, False], ignore_index=True)

# get first 6 genes based on each domain
_domain_genes_df = _group_df[_group_df['domain'].isin([1,4,7])]
_domain_genes_df.to_csv(os.path.join(wk_root, "notebook/results/SVG/ME_147_genes4GO.csv"),index=False)
###
# download GO terms from http://geneontology.org/

In [7]:
d1_gs = _domain_genes_df[_domain_genes_df['domain']==1]['gene'].unique().tolist()
d4_gs = _domain_genes_df[_domain_genes_df['domain']==4]['gene'].unique().tolist()
d7_gs = _domain_genes_df[_domain_genes_df['domain']==7]['gene'].unique().tolist()

In [22]:
print('\n'.join(d1_gs))

Tuba1a
Nnat
Mapt
Stmn2
Tubb2a
Basp1
Uchl1
Gap43
Rtn1
Map1b
Crmp1
Stmn3
Copg2
Ina
Snhg11
Calm1
Ybx1
Mllt11
Gpm6a
Klc1
Nova1
Enc1
Pkia
Calm2
Rab3c
Cpe
Stmn4
Tspyl4
Gnao1
Chl1
Nrxn1
Dner
Ncam1
Rab6b
Cadm2
Rtn3
Reep1
Hmgcs1
Mapre2
Zwint
Ids
Sdc3
Ndn
Rab6a
Gprasp1
Cdc42
Pafah1b1
Serinc1


In [18]:
print('\n'.join(d4_gs))

Krt10
Krt15
Krt77
Lor
Krtdap
Dmkn
Krt4
Perp
S100a6
Emp1
Sbsn
Anxa2
Ahnak


In [19]:
print('\n'.join(d7_gs))

Col1a1
Col1a2
Ibsp
Rplp1
Sparc
Rpsa
Serpinh1
Rpl32
Rplp0
Rps15a
Col5a2
Rps20


In [9]:
GO_d1 = gp.enrichr(
    gene_list=d1_gs, 
    gene_sets= ['GO_Biological_Process_2021', 'GO_Cellular_Component_2021','GO_Molecular_Function_2021'],
    organism='Mouse',
    outdir=None,
)
col = 'Adjusted P-value'
GO_d1_adjp_sort_df = GO_d1.results.sort_values(col)
GO_d1_adjp_sort_df = GO_d1_adjp_sort_df[GO_d1_adjp_sort_df[col]<0.05]
GO_d1_adjp_sort_df.Term = GO_d1_adjp_sort_df.Term.str.split(" \(GO").str[0]

In [10]:
GO_d1_adjp_sort_df.to_csv(os.path.join(wk_root, "notebook/results/SVG/ME_domain1_GO.csv"),index=False)

In [11]:
GO_d4 = gp.enrichr(
    gene_list=d4_gs, 
    gene_sets= ['GO_Biological_Process_2021', 'GO_Cellular_Component_2021','GO_Molecular_Function_2021'],
    organism='Mouse',
    outdir=None,
)
col = 'Adjusted P-value'
GO_d4_adjp_sort_df = GO_d4.results.sort_values(col)
GO_d4_adjp_sort_df = GO_d4_adjp_sort_df[GO_d4_adjp_sort_df[col]<0.05]
GO_d4_adjp_sort_df.Term = GO_d4_adjp_sort_df.Term.str.split(" \(GO").str[0]

In [12]:
GO_d4_adjp_sort_df.to_csv(os.path.join(wk_root, "notebook/results/SVG/ME_domain4_GO.csv"),index=False)

In [13]:
GO_d7 = gp.enrichr(
    gene_list=d7_gs, 
    gene_sets= ['GO_Biological_Process_2021', 'GO_Cellular_Component_2021','GO_Molecular_Function_2021'],
    organism='Mouse',
    outdir=None,
)
col = 'Adjusted P-value'
GO_d7_adjp_sort_df = GO_d7.results.sort_values(col)
GO_d7_adjp_sort_df = GO_d7_adjp_sort_df[GO_d7_adjp_sort_df[col]<0.05]
GO_d7_adjp_sort_df.Term = GO_d7_adjp_sort_df.Term.str.split(" \(GO").str[0]

In [14]:
GO_d7_adjp_sort_df.to_csv(os.path.join(wk_root, "notebook/results/SVG/ME_domain7_GO.csv"),index=False)

# KEGG

In [17]:
KEGG_d1 = gp.enrichr(
    gene_list=d1_gs, 
    gene_sets= ['KEGG_2019_Mouse', 'MSigDB_Hallmark_2020'],
    organism='Mouse',
    outdir=None,
)
col = 'Adjusted P-value'
KEGG_d1_adjp_sort_df = KEGG_d1.results.sort_values(col)
KEGG_d1_adjp_sort_df = KEGG_d1_adjp_sort_df[KEGG_d1_adjp_sort_df[col]<0.05]
KEGG_d1_adjp_sort_df.Term = KEGG_d1_adjp_sort_df.Term.str.split(" \(GO").str[0]

In [18]:
KEGG_d1_adjp_sort_df

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,KEGG_2019_Mouse,Alzheimer disease,4/175,0.000816,0.023136,0,0,10.516215,74.779348,RTN3;MAPT;CALM1;CALM2
1,KEGG_2019_Mouse,Alcoholism,4/199,0.001314,0.023136,0,0,9.210723,61.109522,GNAO1;PKIA;CALM1;CALM2
2,KEGG_2019_Mouse,GnRH signaling pathway,3/90,0.001316,0.023136,0,0,15.222222,100.966097,CDC42;CALM1;CALM2
3,KEGG_2019_Mouse,Rap1 signaling pathway,4/209,0.001573,0.023136,0,0,8.756984,56.522142,CDC42;GNAO1;CALM1;CALM2
4,KEGG_2019_Mouse,Circadian entrainment,3/99,0.001731,0.023136,0,0,13.788889,87.682031,GNAO1;CALM1;CALM2
5,KEGG_2019_Mouse,Melanogenesis,3/100,0.001782,0.023136,0,0,13.646048,86.380955,GNAO1;CALM1;CALM2
6,KEGG_2019_Mouse,Phototransduction,2/27,0.001905,0.023136,0,0,34.655652,217.051981,CALM1;CALM2
85,MSigDB_Hallmark_2020,Myogenesis,4/200,0.001339,0.028110,0,0,9.163265,60.625569,REEP1;GNAO1;PKIA;NCAM1
7,KEGG_2019_Mouse,Neurotrophin signaling pathway,3/121,0.003064,0.032552,0,0,11.205650,64.859874,CDC42;CALM1;CALM2
86,MSigDB_Hallmark_2020,Hedgehog Signaling,2/36,0.003373,0.035418,0,0,25.470588,144.976489,RTN1;CRMP1
